Copyright 2021 Vector Informatik GmbH
SPDX-License-Identifier: MIT 
Contact: khanlian.chung@vector.com

# Notebook for exploring and playing with saliency maps and backprop

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## A few helper functions to load an arbitrary image and generate a simple grey-value heatmap

Some helper function to pre and post processing the inputs.

In [ ]:
def prep_input(path):
    image = tf.image.decode_png(tf.io.read_file(path))
    image = tf.expand_dims(image, axis=0)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [224,224])
    return image

def norm_flat_image(img):
    grads_norm = img[:,:,0]+ img[:,:,1]+ img[:,:,2]
    grads_norm = (grads_norm - tf.reduce_min(grads_norm))/ (tf.reduce_max(grads_norm)- tf.reduce_min(grads_norm))
    return grads_norm
    
def get_lion():
    # Wikipedia lion image https://en.wikipedia.org/wiki/Lion#/media/File:Lion_waiting_in_Namibia.jpg
    # License: CC BY 2.0 https://creativecommons.org/licenses/by/2.0/deed.en
    import urllib.request
    filename = 'lion.jpg'
    img_url = r"https://upload.wikimedia.org/wikipedia/commons/7/73/Lion_waiting_in_Namibia.jpg"
    urllib.request.urlretrieve(img_url, filename)

def plot_maps(img1, img2,vmin=0.3,vmax=0.7, mix_val=2):
    f = plt.figure(figsize=(15,45))
    plt.subplot(1,3,1)
    plt.imshow(img1,vmin=vmin, vmax=vmax, cmap="gray")
    plt.axis("off")
    plt.subplot(1,3,2)
    plt.imshow(img2, cmap = "gray")
    plt.axis("off")
    plt.subplot(1,3,3)
    plt.imshow(img1*mix_val+img2/mix_val, cmap = "gray" )
    plt.axis("off")

This example is built upon a DenseNet201. But you can interchange it any other network. Simpliest way would be: change tf.keras.applications.densenet.DenseNet201() to another network such as tf.keras.applications.resnet.ResNet50(). Don't forget to change the preprocess_input accordingly. If you are interested in the imagenet data, have a look here: https://knowyourdata-tfds.withgoogle.com/#tab=STATS&dataset=imagenet2012


In [ ]:
test_model = tf.keras.applications.densenet.DenseNet201()
#test_model.summary()
get_lion()
img_path = "lion.jpg"
input_img = prep_input(img_path)
input_img = tf.keras.applications.densenet.preprocess_input(input_img)
plt.imshow(norm_flat_image(input_img[0]), cmap = "gray")


In [ ]:
result = test_model(input_img)
max_idx = tf.argmax(result,axis = 1)
tf.keras.applications.imagenet_utils.decode_predictions(result.numpy())


## Guided Backprop:
First, we define our own custom ReLU function

In [ ]:
@tf.custom_gradient
def guidedRelu(x):
    def grad(dy):
        return tf.cast(dy>0,tf.float32)  * tf.cast(x>0,tf.float32) * dy
    return tf.nn.relu(x), grad

Afterwards, we substitute all old, original ReLU functions with our own versions.

In [ ]:
layer_dict = [layer for layer in test_model.layers[1:] if hasattr(layer,'activation')]
for layer in layer_dict:
    if layer.activation == tf.keras.activations.relu:
        layer.activation = guidedRelu
        print("changed")


After setting everything up, we have to explicitly instruct TF to calculate and keep the gradients for the input image input_img.

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(input_img)
    result = test_model(input_img)
    max_score = result[0,max_idx[0]]
grads = tape.gradient(max_score, input_img)

In [ ]:
plot_maps(norm_flat_image(grads[0]), norm_flat_image(input_img[0]))


## If you want to play around with different image windows, you can use napari

In [ ]:
import napari
viewer = napari.view_image(norm_flat_image(grads[0]).numpy())
viewer.add_image(norm_flat_image(input_img[0]).numpy())
